In [34]:
import nibabel as nl
import numpy as np
import pandas as pd
import os, shutil
import matplotlib.pyplot as plt

In [35]:
df = pd.read_csv('df_with_schaefer.csv')
df.head()

,Subject_ID,imaging waves,snr_total,17Networks_RH_TempPar_10,17Networks_RH_TempPar_9,17Networks_RH_TempPar_8,17Networks_RH_TempPar_7,17Networks_RH_TempPar_6,17Networks_RH_TempPar_5,17Networks_RH_TempPar_4,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,sub-0015,ses-wave3,11.837327,-0.331545,0.023124,-0.203593,-0.151195,0.037073,-0.050264,-0.003429,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,sub-0085,ses-wave2,11.131926,-0.044947,-0.216448,-0.198601,-0.222929,-0.005697,-0.206885,0.096881,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520


### list of ALFFs of sub1 and sub2

In [36]:
# opening the text file
lis = []
with open('meants_BNatlas.txt','r') as ALFFout:
   
    # reading each line    
    for line in ALFFout:
   
        # reading each word        
        for ALFF in line.split():
            lis.append(float(ALFF))
            # displaying the words           
            #print(ALFF) 

In [37]:
len(lis)

492

### list of region names in BN Atlas (246 regions)

In [38]:
token = open('BN_Atlas_246_LUT.txt','r')
linestoken=token.readlines()
tokens_column_number = 1
parname=[]
for x in linestoken:
    parname.append(x.split()[tokens_column_number])
token.close()
print(parname)

['A8m_L', 'A8m_R', 'A8dl_L', 'A8dl_R', 'A9l_L', 'A9l_R', 'A6dl_L', 'A6dl_R', 'A6m_L', 'A6m_R', 'A9m_L', 'A9m_R', 'A10m_L', 'A10m_R', 'A9/46d_L', 'A9/46d_R', 'IFJ_L', 'IFJ_R', 'A46_L', 'A46_R', 'A9/46v_L', 'A9/46v_R', 'A8vl_L', 'A8vl_R', 'A6vl_L', 'A6vl_R', 'A10l_L', 'A10l_R', 'A44d_L', 'A44d_R', 'IFS_L', 'IFS_R', 'A45c_L', 'A45c_R', 'A45r_L', 'A45r_R', 'A44op_L', 'A44op_R', 'A44v_L', 'A44v_R', 'A14m_L', 'A14m_R', 'A12/47o_L', 'A12/47o_R', 'A11l_L', 'A11l_R', 'A11m_L', 'A11m_R', 'A13_L', 'A13_R', 'A12/47l_L', 'A12/47l_R', 'A4hf_L', 'A4hf_R', 'A6cdl_L', 'A6cdl_R', 'A4ul_L', 'A4ul_R', 'A4t_L', 'A4t_R', 'A4tl_L', 'A4tl_R', 'A6cvl_L', 'A6cvl_R', 'A1/2/3ll_L', 'A1/2/3ll_R', 'A4ll_L', 'A4ll_R', 'A38m_L', 'A38m_R', 'A41/42_L', 'A41/42_R', 'TE1.0/TE1.2_L', 'TE1.0/TE1.2_R', 'A22c_L', 'A22c_R', 'A38l_L', 'A38l_R', 'A22r_L', 'A22r_R', 'A21c_L', 'A21c_R', 'A21r_L', 'A21r_R', 'A37dl_L', 'A37dl_R', 'aSTS_L', 'aSTS_R', 'A20iv_L', 'A20iv_R', 'A37elv_L', 'A37elv_R', 'A20r_L', 'A20r_R', 'A20il_L', 'A20il

In [39]:
len(parname)

246

### merging regions and their values with dataframe

In [40]:
test = df.set_index('Subject_ID')
inds = test.index
test

,imaging waves,snr_total,17Networks_RH_TempPar_10,17Networks_RH_TempPar_9,17Networks_RH_TempPar_8,17Networks_RH_TempPar_7,17Networks_RH_TempPar_6,17Networks_RH_TempPar_5,17Networks_RH_TempPar_4,17Networks_RH_TempPar_3,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,ses-wave3,11.837327,-0.331545,0.023124,-0.203593,-0.151195,0.037073,-0.050264,-0.003429,-0.147292,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,ses-wave2,11.131926,-0.044947,-0.216448,-0.198601,-0.222929,-0.005697,-0.206885,0.096881,-0.198279,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520


In [41]:
df = pd.DataFrame()
for j, ind in enumerate(inds):
    sub = test.loc[[ind]]
    for i, pars in enumerate(parname):
        if pars not in sub:
            sub.insert(2, pars, np.nan, True)
        sub[pars] = lis[i+j*246]
    df = pd.concat([df, sub])

In [42]:
df

,imaging waves,snr_total,lPFtha_R,lPFtha_L,cTtha_R,cTtha_L,Otha_R,Otha_L,PPtha_R,PPtha_L,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,ses-wave3,11.837327,0.060496,0.225457,-0.025288,0.050415,-0.127054,0.303291,-0.133763,0.192351,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,ses-wave2,11.131926,-0.041604,0.140033,0.041077,-0.176764,-0.084705,0.139243,0.002530,-0.112836,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520


In [44]:
df.drop(df.columns[0:2], axis = 1, inplace = True)
  
df

,lPFtha_R,lPFtha_L,cTtha_R,cTtha_L,Otha_R,Otha_L,PPtha_R,PPtha_L,rTtha_R,rTtha_L,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,0.060496,0.225457,-0.025288,0.050415,-0.127054,0.303291,-0.133763,0.192351,0.627222,0.730700,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,-0.041604,0.140033,0.041077,-0.176764,-0.084705,0.139243,0.002530,-0.112836,0.660913,0.636764,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520


In [27]:
df.to_csv("df_with_schaefer_brainnetome.csv")

In [32]:
for i in range(2,5):
    df.columns.values[i] = df.columns.values[i] + "_Roll Number"

In [33]:
df

,imaging waves,snr_total,lPFtha_RRoll Number_Roll Number_Roll Number,lPFtha_L_Roll Number,cTtha_R_Roll Number,cTtha_L,Otha_R,Otha_L,PPtha_R,PPtha_L,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,ses-wave3,11.837327,0.060496,0.225457,-0.025288,0.050415,-0.127054,0.303291,-0.133763,0.192351,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,ses-wave2,11.131926,-0.041604,0.140033,0.041077,-0.176764,-0.084705,0.139243,0.002530,-0.112836,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
